# Hidden Markov Model for Market Regime Detection
## Quantitative Analysis & Model Validation
### Not complete yet

This notebook provides an  analysis of the Hidden Markov Model (HMM) implementation for market regime detection. We'll explore the model's design decisions, parameter optimization, and performance characteristics through statistical validation techniques.

**Key areas of investigation:**
1. Data preprocessing and feature engineering
2. Model parameter selection and optimization
3. Regime identification and characterization
4. Statistical significance testing
5. Performance across different market conditions
6. Robustness to parameter changes

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('fivethirtyeight')
sns.set_palette("deep")

# Import custom modules
import sys
sys.path.append('../')
from src.improved_hmm_model import ImprovedHMMModel
from src.utils.data_loader import load_data
from src.utils.feature_engineering import add_features, normalize_features
from src.utils.permutation_tests import plot_walk_forward_permutation_results
from src.utils.monte_carlo import plot_monte_carlo_results

# Configure notebook display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
%matplotlib inline

## 1. Data Acquisition and Preprocessing

We'll begin by loading historical price data for multiple assets to ensure our analysis is robust across different market types. We'll examine both equity indices and individual stocks to validate the model's versatility.

In [ ]:
# Define tickers to analyze
tickers = [
    'SPY',    # S&P 500 ETF
    'QQQ',    # Nasdaq 100 ETF
    'AAPL',   # Large cap tech
    'XLE',    # Energy sector ETF
    'GLD'     # Gold ETF (different asset class)
]

# Define date ranges to capture different market regimes
start_date = "2010-01-01"  # Long enough to include multiple market cycles
end_date = date.today().strftime('%Y-%m-%d')

# Load data for each ticker
data_dict = {}
for ticker in tickers:
    print(f"Loading data for {ticker}...")
    data = load_data(ticker, start_date, end_date, use_local=True)
    data_dict[ticker] = data
    
# Display summary statistics for each dataset
for ticker, data in data_dict.items():
    print(f"\n{ticker} Data Summary:")
    print(f"Date Range: {data.index.min().date()} to {data.index.max().date()}")
    print(f"Trading Days: {len(data)}")
    print(f"Missing Values: {data.isna().sum().sum()}")
    
# Select SPY as our primary analysis dataset
spy_data = data_dict['SPY']
spy_data.head()

### 1.1 Data Quality Analysis

Before proceeding with modeling, we need to ensure our data is clean and suitable for analysis. Let's examine the data for anomalies, missing values, and other potential issues.

In [ ]:
# Function to analyze data quality
def analyze_data_quality(data, ticker):
    # Check for missing values
    missing = data.isna().sum()
    
    # Check for outliers in returns
    returns = data['Close'].pct_change()
    outliers = returns[abs(returns) > returns.std() * 3]
    
    # Check for gaps in trading days
    date_diffs = data.index.to_series().diff().dt.days
    gaps = date_diffs[date_diffs > 3]  # More than 3 days between observations
    
    # Visualize price history
    plt.figure(figsize=(14, 7))
    plt.plot(data.index, data['Close'])
    plt.title(f'{ticker} Price History')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Visualize return distribution
    plt.figure(figsize=(14, 7))
    sns.histplot(returns.dropna(), kde=True)
    plt.axvline(0, color='r', linestyle='--')
    plt.title(f'{ticker} Daily Return Distribution')
    plt.xlabel('Daily Return')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Return summary
    return {
        'missing_values': missing,
        'outliers': len(outliers),
        'outlier_dates': outliers.index.tolist(),
        'gaps': len(gaps),
        'gap_dates': gaps.index.tolist()
    }

# Analyze SPY data quality
spy_quality = analyze_data_quality(spy_data, 'SPY')
print("Data Quality Analysis:")
print(f"Missing Values: {spy_quality['missing_values'].sum()}")
print(f"Outliers: {spy_quality['outliers']}")
print(f"Gaps in Trading Days: {spy_quality['gaps']}")

### 1.2 Feature Engineering Analysis

The HMM model relies on carefully engineered features to detect market regimes. Let's examine the features created by our feature engineering pipeline and assess their predictive power.

In [ ]:
# Add features to SPY data
spy_features = add_features(spy_data)

# Display available features
print("Available Features:")
feature_columns = [col for col in spy_features.columns if col not in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
print(feature_columns)

# Analyze feature correlations
correlation_matrix = spy_features[feature_columns].corr()

# Plot correlation heatmap
plt.figure(figsize=(16, 14))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Analyze feature distributions
plt.figure(figsize=(16, 12))
for i, feature in enumerate(feature_columns[:12]):  # Plot first 12 features
    plt.subplot(4, 3, i+1)
    sns.histplot(spy_features[feature].dropna(), kde=True)
    plt.title(feature)
    plt.tight_layout()
plt.show()

### 1.3 Feature Selection Analysis

The model uses SelectKBest with f_regression to select the most informative features. Let's analyze which features are consistently selected and their predictive power.

In [ ]:
# Initialize HMM model with feature selection enabled
hmm_model = ImprovedHMMModel(feature_selection=True, k_features=8)

# Create target variable (5-day forward returns)
spy_features['Forward_Return_5d'] = spy_features['Close'].pct_change(5).shift(-5)

# Prepare features for selection
X = spy_features[feature_columns].dropna()
y = spy_features['Forward_Return_5d'].loc[X.index].dropna()

# Align X and y
common_idx = X.index.intersection(y.index)
X = X.loc[common_idx]
y = y.loc[common_idx]

# Perform feature selection
from sklearn.feature_selection import SelectKBest, f_regression
selector = SelectKBest(f_regression, k=8)
X_selected = selector.fit_transform(X, y)

# Get feature scores
feature_scores = pd.DataFrame({
    'Feature': X.columns,
    'F_Score': selector.scores_,
    'P_Value': selector.pvalues_
})
feature_scores = feature_scores.sort_values('F_Score', ascending=False)

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x='F_Score', y='Feature', data=feature_scores)
plt.title('Feature Importance (F-Score)')
plt.tight_layout()
plt.show()

# Display selected features
selected_features = feature_scores.head(8)['Feature'].tolist()
print("Selected Features:")
for i, feature in enumerate(selected_features, 1):
    print(f"{i}. {feature}")

## 2. HMM Model Parameter Optimization

The HMM model has several key parameters that need to be optimized, including the number of states and covariance type. Let's analyze the parameter selection process and validate the chosen parameters.

In [ ]:
# Function to evaluate HMM parameters
def evaluate_hmm_parameters(data, n_components_range, covariance_types):
    results = []
    
    # Add features
    data_with_features = add_features(data)
    
    # Prepare HMM features
    hmm_model = ImprovedHMMModel(feature_selection=False)  # Disable feature selection for this test
    hmm_features = hmm_model.prepare_hmm_features(data_with_features)
    
    # Normalize features
    normalized_features, _ = normalize_features(hmm_features.values, fit=True)
    
    # Test different parameters
    from hmmlearn.hmm import GaussianHMM
    from sklearn.metrics import silhouette_score
    
    for n_components in n_components_range:
        for covariance_type in covariance_types:
            
            try:
                # Train model
                model = GaussianHMM(
                    n_components=n_components,
                    covariance_type=covariance_type,
                    n_iter=1000,
                    random_state=42)
                model.fit(normalized_features)
                
                # Predict states
                states = model.predict(normalized_features)
                
                # Calculate silhouette score
                silhouette = silhouette_score(normalized_features, states)
                
                # Calculate BIC
                bic = model.bic(normalized_features)
                
                # Calculate log likelihood
                log_likelihood = model.score(normalized_features)
                
                # Store results
                results.append({
                    'n_components': n_components,
                    'covariance_type': covariance_type,
                    'silhouette': silhouette,
                    'bic': bic,
                    'log_likelihood': log_likelihood
                })
                
                print(f"n_components={n_components}, covariance_type={covariance_type}, silhouette={silhouette:.4f}, BIC={bic:.2f}")
            except Exception as e:
                print(f"Error with n_components={n_components}, covariance_type={covariance_type}: {e}")
    
    return pd.DataFrame(results)

# Evaluate HMM parameters on SPY data
param_results = evaluate_hmm_parameters(
    spy_data, 
    n_components_range=range(2, 6),
    covariance_types=["full", "diag", "tied", "spherical"]
)

# Plot parameter evaluation results
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot silhouette scores
for cov_type in param_results['covariance_type'].unique():
    subset = param_results[param_results['covariance_type'] == cov_type]
    axes[0].plot(subset['n_components'], subset['silhouette'], marker='o', label=cov_type)
axes[0].set_title('Silhouette Score by Number of States')
axes[0].set_xlabel('Number of States')
axes[0].set_ylabel('Silhouette Score')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot BIC
for cov_type in param_results['covariance_type'].unique():
    subset = param_results[param_results['covariance_type'] == cov_type]
    axes[1].plot(subset['n_components'], subset['bic'], marker='o', label=cov_type)
axes[1].set_title('BIC by Number of States (Lower is Better)')
axes[1].set_xlabel('Number of States')
axes[1].set_ylabel('BIC')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot log likelihood
for cov_type in param_results['covariance_type'].unique():
    subset = param_results[param_results['covariance_type'] == cov_type]
    axes[2].plot(subset['n_components'], subset['log_likelihood'], marker='o', label=cov_type)
axes[2].set_title('Log Likelihood by Number of States (Higher is Better)')
axes[2].set_xlabel('Number of States')
axes[2].set_ylabel('Log Likelihood')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find optimal parameters
best_silhouette = param_results.loc[param_results['silhouette'].idxmax()]
best_bic = param_results.loc[param_results['bic'].idxmin()]
best_likelihood = param_results.loc[param_results['log_likelihood'].idxmax()]

print("\nOptimal Parameters:")
print(f"Best by Silhouette: {best_silhouette['n_components']} states, {best_silhouette['covariance_type']} covariance")
print(f"Best by BIC: {best_bic['n_components']} states, {best_bic['covariance_type']} covariance")
print(f"Best by Log Likelihood: {best_likelihood['n_components']} states, {best_likelihood['covariance_type']} covariance")

### 2.1 Cross-Validation of HMM Parameters

To ensure our parameter selection is robust, let's perform time series cross-validation to validate the stability of our parameter choices across different time periods.

In [ ]:
# Perform time series cross-validation for HMM parameters
def ts_cross_validate_hmm(data, n_splits=5):
    from sklearn.model_selection import TimeSeriesSplit
    
    # Create time series splits
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    # Add features
    data_with_features = add_features(data)
    
    # Prepare HMM features
    hmm_model = ImprovedHMMModel(feature_selection=True, k_features=8)
    hmm_features = hmm_model.prepare_hmm_features(data_with_features)
    
    # Create target for feature selection
    y = data_with_features['Close'].pct_change(5).shift(-5)
    
    # Results storage
    cv_results = []
    
    # Perform cross-validation
    for i, (train_idx, test_idx) in enumerate(tscv.split(hmm_features)):
        print(f"\nFold {i+1}/{n_splits}")
        
        # Split data
        train_features = hmm_features.iloc[train_idx]
        train_y = y.iloc[train_idx]
        test_features = hmm_features.iloc[test_idx]
        test_y = y.iloc[test_idx]
        
        # Train HMM model
        fold_model = ImprovedHMMModel(n_components_range=(2, 5), n_splits=3)
        
        try:
            # Fit model on training data
            train_data_fold = data.iloc[train_idx]
            fold_model.fit(train_data_fold)
            
            # Get optimal parameters for this fold
            optimal_params = fold_model.best_params
            
            # Store results
            cv_results.append({
                'fold': i+1,
                'train_size': len(train_idx),
                'test_size': len(test_idx),
                'n_components': optimal_params['n_components'],
                'covariance_type': optimal_params['covariance_type'],
                'bull_state': fold_model.bull_state,
                'bear_state': fold_model.bear_state,
                'neutral_state': fold_model.neutral_state
            })
            
            print(f"Optimal parameters: {optimal_params['n_components']} states, {optimal_params['covariance_type']} covariance")
            print(f"Bull state: {fold_model.bull_state}, Bear state: {fold_model.bear_state}, Neutral state: {fold_model.neutral_state}")
            
        except Exception as e:
            print(f"Error in fold {i+1}: {e}")
    
    return pd.DataFrame(cv_results)

# Run time series cross-validation
cv_results = ts_cross_validate_hmm(spy_data, n_splits=5)

# Analyze cross-validation results
print("\nCross-Validation Results:")
print(cv_results)

# Count parameter frequency
n_components_counts = cv_results['n_components'].value_counts()
covariance_counts = cv_results['covariance_type'].value_counts()

# Plot parameter frequency
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

n_components_counts.plot(kind='bar', ax=ax1)
ax1.set_title('Frequency of Optimal Number of States')
ax1.set_xlabel('Number of States')
ax1.set_ylabel('Frequency')

covariance_counts.plot(kind='bar', ax=ax2)
ax2.set_title('Frequency of Optimal Covariance Type')
ax2.set_xlabel('Covariance Type')
ax2.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 3. Market Regime Identification

One of the key aspects of the HMM model is its ability to identify different market regimes (bull, bear, neutral). Let's analyze how these regimes are characterized and validate their economic interpretation.

In [ ]:
# Train the HMM model on the full dataset
hmm_model = ImprovedHMMModel()
hmm_model.fit(spy_data)

# Generate predictions
predictions = hmm_model.predict(spy_data)

# Plot the identified regimes
hmm_model.plot_states(predictions, title='SPY Market Regimes')

# Analyze regime characteristics
regime_stats = {}
for state in range(hmm_model.best_params['n_components']):
    regime_data = predictions[predictions['Predicted_State'] == state]
    
    # Skip if no data for this state
    if len(regime_data) == 0:
        continue
    
    # Determine regime type
    if state == hmm_model.bull_state:
        regime_type = 'Bull'
    elif state == hmm_model.bear_state:
        regime_type = 'Bear'
    elif state == hmm_model.neutral_state:
        regime_type = 'Neutral'
    else:
        regime_type = f'State {state}'
    
    # Calculate statistics
    regime_stats[regime_type] = {
        'count': len(regime_data),
        'pct_of_total': len(regime_data) / len(predictions) * 100,
        'avg_daily_return': regime_data['Log_Return'].mean() * 100,
        'volatility': regime_data['Log_Return'].std() * 100 * np.sqrt(252),
        'sharpe': (regime_data['Log_Return'].mean() / regime_data['Log_Return'].std() * np.sqrt(252)) if regime_data['Log_Return'].std() > 0 else 0,
        'win_rate': (regime_data['Log_Return'] > 0).mean() * 100,
        'avg_volume': regime_data['Volume'].mean() if 'Volume' in regime_data.columns else None,
        'avg_volatility': regime_data['Return_Volatility'].mean() * 100 if 'Return_Volatility' in regime_data.columns else None
    }

# Display regime statistics
regime_stats_df = pd.DataFrame(regime_stats).T
print("Regime Statistics:")
display(regime_stats_df)

# Plot regime return distributions
hmm_model.plot_state_distributions(predictions)

### 3.1 Regime Transition Analysis

Let's analyze the transition probabilities between different market regimes to understand the persistence of each regime and the likelihood of transitions.

In [ ]:
# Extract transition matrix from the model
transition_matrix = hmm_model.best_model.transmat_

# Create DataFrame for better visualization
states = range(hmm_model.best_params['n_components'])
transition_df = pd.DataFrame(transition_matrix, index=states, columns=states)

# Map state numbers to regime names
regime_names = {}
for state in states:
    if state == hmm_model.bull_state:
        regime_names[state] = f"Bull (State {state})"
    elif state == hmm_model.bear_state:
        regime_names[state] = f"Bear (State {state})"
    elif state == hmm_model.neutral_state:
        regime_names[state] = f"Neutral (State {state})"
    else:
        regime_names[state] = f"State {state}"

# Rename index and columns
transition_df.index = [regime_names[state] for state in states]
transition_df.columns = [regime_names[state] for state in states]

# Display transition matrix
print("Transition Probability Matrix:")
display(transition_df)

# Plot transition matrix as heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(transition_df, annot=True, cmap='YlGnBu', fmt='.3f', linewidths=0.5)
plt.title('Regime Transition Probabilities')
plt.tight_layout()
plt.show()

# Calculate regime persistence
persistence = {regime: transition_df.loc[regime, regime] for regime in transition_df.index}
persistence_df = pd.DataFrame(list(persistence.items()), columns=['Regime', 'Persistence'])
persistence_df['Expected_Duration_Days'] = 1 / (1 - persistence_df['Persistence'])

# Plot regime persistence
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='Regime', y='Expected_Duration_Days', data=persistence_df)
plt.title('Expected Duration of Each Market Regime')
plt.ylabel('Expected Duration (Trading Days)')
plt.xticks(rotation=45)

# Add value labels
for i, v in enumerate(persistence_df['Expected_Duration_Days']):
    ax.text(i, v + 0.5, f"{v:.1f}", ha='center')

plt.tight_layout()
plt.show()

### 3.2 Historical Regime Analysis

Let's examine how the identified regimes align with known historical market events to validate the model's ability to capture meaningful market states.

In [ ]:
# Define major market events
market_events = [
    {'date': '2010-05-06', 'event': 'Flash Crash', 'type': 'negative'},
    {'date': '2011-08-05', 'event': 'US Credit Downgrade', 'type': 'negative'},
    {'date': '2015-08-24', 'event': 'China Slowdown Fears', 'type': 'negative'},
    {'date': '2016-06-24', 'event': 'Brexit Vote', 'type': 'negative'},
    {'date': '2018-02-05', 'event': 'Volatility Spike', 'type': 'negative'},
    {'date': '2020-03-23', 'event': 'COVID-19 Market Bottom', 'type': 'positive'},
    {'date': '2022-01-03', 'event': 'Fed Tightening Begins', 'type': 'negative'},
    {'date': '2023-03-10', 'event': 'Silicon Valley Bank Collapse', 'type': 'negative'}
]

# Convert dates to datetime
for event in market_events:
    event['date'] = pd.to_datetime(event['date'])

# Function to find the regime around a specific date
def get_regime_around_event(predictions, event_date, window=10):
    # Find the closest date in the predictions
    if event_date not in predictions.index:
        closest_date = predictions.index[predictions.index.get_indexer([event_date], method='nearest')[0]]
    else:
        closest_date = event_date
    
    # Get index location
    event_idx = predictions.index.get_loc(closest_date)
    
    # Get window around event
    start_idx = max(0, event_idx - window)
    end_idx = min(len(predictions), event_idx + window + 1)
    
    # Get data around event
    event_window = predictions.iloc[start_idx:end_idx]
    
    # Count regimes in window
    regime_counts = event_window['Market_Regime'].value_counts()
    dominant_regime = regime_counts.idxmax() if not regime_counts.empty else 'Unknown'
    
    # Get regime on event day
    event_day_regime = predictions.loc[closest_date, 'Market_Regime'] if closest_date in predictions.index else 'Unknown'
    
    return {
        'event_date': event_date,
        'closest_date': closest_date,
        'event_day_regime': event_day_regime,
        'dominant_regime': dominant_regime,
        'regime_counts': regime_counts.to_dict()
    }

# Analyze regimes around market events
event_regimes = []
for event in market_events:
    regime_info = get_regime_around_event(predictions, event['date'])
    event_regimes.append({
        'Event': event['event'],
        'Date': event['date'],
        'Type': event['type'],
        'Event Day Regime': regime_info['event_day_regime'],
        'Dominant Regime': regime_info['dominant_regime']
    })

# Display event regimes
event_regimes_df = pd.DataFrame(event_regimes)
print("Market Regimes During Major Events:")
display(event_regimes_df)

# Calculate accuracy of regime identification
correct_identifications = sum(
    (row['Type'] == 'negative' and row['Dominant Regime'] == 'Bear') or
    (row['Type'] == 'positive' and row['Dominant Regime'] == 'Bull')
    for row in event_regimes
)
accuracy = correct_identifications / len(event_regimes) * 100

print(f"\nAccuracy of Regime Identification for Known Events: {accuracy:.2f}%")

# Plot regimes with market events
plt.figure(figsize=(16, 8))

# Plot price
plt.plot(predictions.index, predictions['Close'], color='black', linewidth=1.5)

# Color background by regime
for state in range(hmm_model.best_params['n_components']):
    mask = predictions['Predicted_State'] == state
    if state == hmm_model.bull_state:
        color = 'green'
        alpha = 0.2
        label = 'Bull Market'
    elif state == hmm_model.bear_state:
        color = 'red'
        alpha = 0.2
        label = 'Bear Market'
    elif state == hmm_model.neutral_state:
        color = 'blue'
        alpha = 0.2
        label = 'Neutral Market'
    else:
        color = 'gray'
        alpha = 0.2
        label = f'State {state}'
    
    # Find contiguous regions
    regions = []
    start_idx = None
    
    for idx, val in enumerate(mask):
        if val and start_idx is None:
            start_idx = idx
        elif not val and start_idx is not None:
            regions.append((start_idx, idx - 1))
            start_idx = None
    
    # Add the last region if it extends to the end
    if start_idx is not None:
        regions.append((start_idx, len(mask) - 1))
    
    # Shade each region
    for start, end in regions:
        plt.axvspan(predictions.index[start], predictions.index[end], 
                  alpha=alpha, color=color, label=label if start == regions[0][0] else None)

# Add market events
for event in market_events:
    if event['type'] == 'negative':
        color = 'red'
        marker = 'v'
    else:
        color = 'green'
        marker = '^'
    
    # Find closest date in data
    if event['date'] not in predictions.index:
        closest_date = predictions.index[predictions.index.get_indexer([event['date']], method='nearest')[0]]
    else:
        closest_date = event['date']
    
    # Get price at event
    price = predictions.loc[closest_date, 'Close']
    
    # Plot event marker
    plt.scatter(closest_date, price, color=color, marker=marker, s=100, zorder=5)
    
    # Add event label
    plt.annotate(event['event'], 
                xy=(closest_date, price),
                xytext=(10, 0),
                textcoords='offset points',
                ha='left',
                va='center',
                fontsize=9,
                rotation=45)

plt.title('Market Regimes and Major Events')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## 4. Trading Strategy Performance

Now let's evaluate the performance of trading strategies based on the HMM model's regime predictions.

In [ ]:
# Calculate strategy returns
predictions['Strategy_Return'] = predictions['Signal'].shift(1) * predictions['Log_Return']
predictions['BuyHold_Return'] = predictions['Log_Return']

# Calculate cumulative returns
predictions['Strategy_Cumulative'] = np.exp(predictions['Strategy_Return'].cumsum()) - 1
predictions['BuyHold_Cumulative'] = np.exp(predictions['BuyHold_Return'].cumsum()) - 1

# Plot cumulative returns
plt.figure(figsize=(14, 7))
plt.plot(predictions.index, predictions['Strategy_Cumulative'] * 100, label='HMM Strategy', linewidth=2)
plt.plot(predictions.index, predictions['BuyHold_Cumulative'] * 100, label='Buy & Hold', linewidth=2, alpha=0.7)
plt.title('Cumulative Returns: HMM Strategy vs Buy & Hold')
plt.xlabel('Date')
plt.ylabel('Cumulative Return (%)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()
